<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
کاریابی کوئرا 2
</font>
</h1>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
در پروژه قبل شما با استفاده از داده های بخش کاریابی کوئرا سعی کردید یک سیستم پیشنهاد دهنده مبتنی بر کاربر بسازید. و البته موفق هم شدید و دیدید با توجه به سوابق ارسال رزومه های خود کاربر، خروجی سیستم شما میتوانست آگهی های مشابه را به خوبی پیشنهاد دهد. در این تمرین میخواهیم همین کار را با کمک رویکرد مبتنی بر محصول یا مبتنی بر آیتم انجام دهیم. در تمرین قبل ما از ماتریس کاربر-آیتم به ماتریس کاربر-کاربر رسیدیم یعنی با توجه به سوابق ارسال رزومه ها، حساب کردیم که کدام کاربران به یکدیگر شبیه تر اند و با استفاده از این ماتریس آگهی جدید پیشنهاد دادیم. در این تمرین شما باید ماتریس آیتم-آیتم بسازید یعنی از دل ماتریس کاربر-آیتم میزان شباهت آیتم ها به یکدیگر را محاسبه کنید. که خب نسبت به ماتریس کاربر-کاربر ماتریس تُنُک تر یا sparseتر است  و چگال کردن این ماتریس نیازمند فضای حافظه بیشتری است اما خب مزایای خاص خود را نسبت به سیستم UBCF دارد که در درسنامه ها به آنها اشاره شده است.
</font>
</p>


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
ابتدا کتابخانه های مورد نیاز و مجموعه داده <code>quera-karyabi.csv</code> را به نوت بوک اضافه کن.
</font>
</p>


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy

df = pd.read_csv('./quera-karyabi.csv')


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
ابعاد داده ها را نمایش بده.
</font>
</p>

In [2]:
display(df)

,Applicant ID,Source,Order,Developer State,Seen,Job ID,Job Title,Careers Job - Job → Level,Careers Job - Job → Offers Remote,Careers Job - Job → Minimum Salary,Careers Job - Job → Maximum Salary,Careers Job - Job → Hit Count,Accounts City - City → Name,Careers Disqualifyreason - Disqualify Reason → Name,Job_ID_Title
0,4129056.0,1,0,25,True,280920,توسعه‌دهنده‌ی Front-end,NaN,False,NaN,NaN,0,NaN,NaN,280920 - توسعه‌دهنده‌ی Front-end
1,6696432.0,2,0,30,True,293544,توسعه‌دهنده Back-end,junior,True,8000000.0,12000000.0,0,تهران,عدم تناسب با موقعیت شغلی,293544 - توسعه‌دهنده Back-end
2,4823712.0,2,4,20,True,68232,توسعه‌دهنده React,senior,False,8000000.0,12000000.0,0,مشهد,NaN,68232 - توسعه‌دهنده React
3,8995152.0,2,2,20,True,509400,Software Engineer (Front-end),junior,True,NaN,NaN,2092,تهران,NaN,509400 - Software Engineer (Front-end)
4,9651120.0,2,5,30,True,423384,برنامه‌نویس ارشد .NET,senior,True,NaN,NaN,423,آمل,عدم تناسب با موقعیت شغلی,423384 - برنامه‌نویس ارشد .NET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,3435024.0,2,217,10,False,556128,Junior Front-end Developer (React),junior,False,NaN,NaN,2415,تبریز,NaN,556128 - Junior Front-end Developer (React)
199996,1023552.0,1,1,30,True,333384,Test Engineer,senior,True,NaN,NaN,4,NaN,مهارت یا تجربه ناکافی,333384 - Test Engineer
199997,4027344.0,2,34,10,False,80808,کارآموز BlockChain Development,junior,False,NaN,NaN,0,فردیس,NaN,80808 - کارآموز BlockChain Development
199998,1398336.0,1,12,10,False,269280,برنامه‌نویس C,junior,False,12000000.0,NaN,0,NaN,NaN,269280 - برنامه‌نویس C


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
با استفاده از pandas تعداد داده های null در هر ستون را نمایش بده.
</font>
</p>

In [3]:
df.shape

(200000, 15)

In [4]:
df.isnull().sum()

,0
Applicant ID,4200
Source,0
Order,0
Developer State,0
Seen,0
Job ID,0
Job Title,0
Careers Job - Job → Level,13833
Careers Job - Job → Offers Remote,0
Careers Job - Job → Minimum Salary,150958


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت اول
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
تعداد ستون های زیاد دیتاست به ما اطلاعات بیشتری از جزئیات فرصت شغلی می دهند. اما در نهایت در سیستم های پالایش جمعی ما باید به ماتریس کاربر-آیتم برسیم چرا که قرار هست با توجه به شباهت بین کاربران و اینکه به چه شغلی رزومه فرستاده اند، تصمیم گیری و توصیه کنیم پس دیگر ستون های این دیتاست را نیاز نداریم. با توجه به خروجی بالا و اینکه تعداد نسبتا زیادی از ستون <code>	Applicant ID</code> خالی هستند باید در سلول زیر سطرهایی که حاوی این مقادیر null هستند را حذف کنید. (فقط سطرهایی که ستون applicant ID انها null است.)
</font>
</p>


In [5]:
df = df[df['Applicant ID'].notnull()]

In [6]:
df.shape

(195800, 15)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
نکته
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
اگر ستون های Job ID یا Job Title هم مقادیر خالی داشتند آنهارا هم نیز حذف یا پر میکردیم.
</font>
</p>


In [7]:
df = df[df['Job ID'].notnull()]
df = df[df['Job Title'].notnull()]

In [8]:
df.shape

(195800, 15)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
مشابه پروژه  2، سلول زیر را اجرا کنید تا ستون Job Domain به داده ها افزوده شود.
</font>
</p>

In [9]:
normalization_map = {
    # --- Roles / نقش‌ها ---
    "برنامه نویسی": "developer",
    "برنامه‌نویسی": "developer",
    "برنامه نویس": "developer",
    "برنامه‌نویس": "developer",
    "توسعه دهنده": "developer",
    "توسعه‌دهنده": "developer",
    "توسعه‌دهنده‌ی": "developer",
    "مهندس نرم افزار": "software engineer",
    "مهندس نرم‌افزار": "software engineer",
    "کارشناس نرم افزار": "software specialist",
    "کارشناس نرم‌افزار": "software specialist",
    "توسعه دهنده نرم افزار": "software developer",
    "توسعه‌دهنده نرم‌افزار": "software developer",
    "پشتیبان": "support",
    "تحلیلگر": "analyst",
    "تحلیل گر": "analyst",
    "طراح": "designer",
    "مدیر": "manager",
    "متخصص": "specialist",
    "کارشناس": "specialist",

    # --- Levels / سطوح ---
    "ارشد": "senior",
    "سنیور": "senior",
    "جونیور": "junior",
    "کارآموز": "intern",
    "کاراموز": "intern",
    "تازه کار": "junior",
    "تازهکار": "junior",
    "میدلول": "mid-level",
    "mid level": "mid-level",

    # --- Technologies & Fields / تکنولوژی‌ها و حوزه‌ها ---
    "تحلیل":" analysis",
    "تحلیلگر داده": "data analyst",
    "ساینس": "science",
    "ساینتیست": "scientist",
    "وب": "web",
    "لاراول": "laravel",
    "نرم‌افزار":"software",
    "جنگو":"django",
    "پایتون": "python",
    "جاوااسکریپت": "javascript",
    "جاوا اسکریپت": "javascript",
    "پی اچ پی": "php",
    "پی‌اچ‌پی": "php",
    "سی شارپ": "c#",
    "سی‌شارپ": "c#",
    "دات نت": ".net",
    "دات‌نت": ".net",
    "فرانت اند": "frontend",
    "فرانت‌اند": "frontend",
    "بک اند": "backend",
    "بک‌اند": "backend",
    "فول استک": "fullstack",
    "دواپس": "devops",
    "دوآپس": "devops",
    "هوش مصنوعی": "artificial intelligence",
    "یادگیری ماشین": "machine learning",
    "دیتا": "data",
    "داده": "data",
    "ساینتیست": "scientist",
    "دانشمند": "scientist",
    "علم داده": "data science",
    "اندروید": "android",
    "آی او اس": "ios",
    "آی‌او‌اس": "ios",
    "موبایل": "mobile",
    "امنیت": "security",
    "شبکه": "network",
    "اس کیو ال": "sql",
    "پایگاه داده": "database",
    "داده":"data",
    "دورکاری":"intern",

    "-":"",
    # --- Character Normalization / نرمال‌سازی کاراکترها ---
    "ك": "ک", # کاف عربی به فارسی
    "ي": "ی", # یای عربی به فارسی
}

import re

def normalize_title(title):
    if not isinstance(title, str):
        return "" # یا هر مقدار پیش‌فرض دیگر برای داده‌های غیرمتنی

    title = title.lower()
    for key, value in normalization_map.items():
        title = title.replace(key, value)
    title = re.sub(r'‌?ی(?=\\s*[a-zA-Z])', '', title)
    title = re.sub(r'\s+', ' ', title).strip()

    return title

original_title_column = 'Job Title'
normalized_title_column = 'normal_title'

# اعمال تابع روی ستون و ساخت ستون جدید
df[normalized_title_column] = df[original_title_column].apply(normalize_title)

def get_job_domain(title):
    """
    عنوان شغلی را به یک حوزه اصلی (Domain) دسته‌بندی می‌کند.
    """
    title = str(title).lower()

    ai_data_keywords = ['data scientist', 'machine learning', 'یادگیری ماشین', 'هوش مصنوعی', 'artificial intelligence', 'تحلیلگر داده','تحلیل', 'data analyst','data engineer', 'مهندس داده', 'data science', 'علم داده', 'داده کاوی', 'big data', 'بیگ دیتا']
    backend_keywords = ['backend', 'back-end', 'بک اند', 'django', 'flask','net.', 'fastapi', 'node.js', 'php', 'laravel', 'c#','back end', '.net', 'java', 'spring', 'go','python', 'python developer', 'ruby', 'ruby on rails', 'rails','.net','nodejs']
    frontend_keywords = ['frontend', 'front-end', 'فرانت اند', 'react', 'vue', 'angular', 'javascript','web', 'front end']
    android_keywords = ['android', 'اندروید', 'kotlin', 'کاتلین', 'jetpack', 'compose', 'موبایل','flutter','ios']
    uiux_keywords = ['ui','ux','uiux','designer','دیزاینر','figma']


    if any(keyword in title for keyword in ai_data_keywords):
        return 'AI_Data'
    elif any(keyword in title for keyword in android_keywords):
        return 'Android'
    elif any(keyword in title for keyword in backend_keywords):
        return 'Backend'
    elif any(keyword in title for keyword in frontend_keywords):
        return 'Frontend'
    elif any(keyword in title for keyword in uiux_keywords):
        return 'UIUX'
    else:
        return 'Other'


df['Job Domain'] = df['normal_title'].apply(get_job_domain)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت دوم
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
باید دو ستون جدید ایجاد کنید. هر ردیف ما نشان دهنده یک تعامل است یعنی شخصی به این آیدی به یک آگهی رزومه فرستاده است. پس یک ستون جدید به نام <code>interaction</code> ایجاد کنید. تمام مقادیر ستون ایجاد شده را برابر 1 قرار دهید. ستون بعدی ترکیب دو ستون <code>Job ID</code> و <code>Job Title</code> است. هدف ما در نهایت ستون <code>Job ID</code> است اما از انجا که خود IDها در مرحله نهایی گنگ و نامفهوم هستند ما ستون عنوان شغلی را هم به آن وصل میکنیم. اما همچنان یکتا بودن اگهی ها با توجه به حضور ID حفظ شده.
<br>
هدف از ایجاد ستون <code>Job_ID_Title</code> صرفا دید و بینش نسبت به سیستم است و میتوان با همان ستون <code>Job ID</code> سیستم IBCF را پیاده سازی کرد اما خب خیلی نمیتوان متوجه شد اینکه «شباهت بین دو آیدی 814521 و 745693 برابر 0.7 است» یعنی چه. به همین علت ما عنوان شغلی را نیز به آیدی اضافه میکنیم.
</font>
</p>


In [10]:
df['interaction'] = 1

df['Job_ID_Title'] = df['Job ID'].astype(str) + ' ' + df['normal_title'].astype(str)


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت سوم
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
با استفاده از متد <code>pivot_table</code> کتابخانه <code>Pandas</code> ماتریس کاربر-آیتم خود را بسازید. ستون های این ماتریس موقعیت شغلی های مختلف هستند که در قسمت قبل ستون آن را ساختیم. و سطر این ماتریس، آیدی کاربران است. همچنین اگر ارتباطی واقعا بوده باشد مقدار 1 و اگر تعاملی نباشد بین اگهی و کاربر، مقدار صفر قرار دهید.
</font>
</p>


In [11]:
user_item_matrix = df.pivot_table(index='Applicant ID', columns='Job_ID_Title', values='interaction', fill_value=0)


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
ماتریس کاربر-آیتم خود را نمایش دهید.
</font>
</p>


In [14]:
df['Applicant ID'].nunique()

29570

In [15]:
df['Job ID'].nunique()

13799

In [12]:
print(user_item_matrix.shape)
user_item_matrix.head(15)

(29570, 13799)


Job_ID_Title,100008 specialist artificial intelligence (nlp),100032 specialist کنترل پروژه,100056 developer senior frontend,100104 developer‌ی frontend,100128 frontend developer (reactjs),100152 developer‌ی frontend,100224 specialist اسکرام مستر,100272 designer web,10032 developer (آقا),100368 developer frontend,...,99792 specialist devops,99816 web designer,9984 developer android,99840 designer رابط کاربری ui/ux,99864 specialist تولید محتوا,99888 manager محصول,99912 developer flutter,99936 senior backend developer (python),99960 (backend developer (asp.net,99984 (frontend developer (reactjs
Applicant ID,,,,,,,,,,,,,,,,,,,,,
48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
192.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1488.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2160.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4992.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت چهارم
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
برای بالاتر بردن کیفیت داده ها در سیستم های پیشنهاد دهنده، یکی از کارهایی که می توانیم بکنیم فیلتر کردن داده هاست. برای مثال در این دیتاست از شما میخواهیم که در سلول زیر شغل هایی که کمتر از 15 درخواست برای آنها ثبت شده است را حذف کنید. و در ادامه کاربرانی که به کمتر از 10 آگهی درخواست داده اند نیز حذف شود. این کار هم باعث کاهش ابعاد دیتاست و حجم محاسبات میشود و هم باعث میشود نویز و داده های نادر حذف شوند که البته مزایا و معایب خود را دارد.
</font>
</p>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
<span style="color:green"><b>راهنمایی:</b></span>
ترتیب حذف ردیف ها اهمیت دارد. همانطور که گفته شد، ابتدا آگهی ها را فیتلر کنید و سپس کاربران.
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
<span style="color:green"><b>راهنمایی:</b></span>
در نهایت دوباره ماتریس را در متفیر <code>user_item_matrix</code> ذخیره کنید تا به مشکل کمبود حافظه و پر شدن RAM بر نخورید.
</font>
</p>



In [17]:
min_apps_per_job = 15
min_jobs_per_user = 10

df = df[df['Job ID'].map(df['Job ID'].value_counts()) >= min_apps_per_job]
df = df[df['Applicant ID'].map(df['Applicant ID'].value_counts()) >= min_jobs_per_user]

In [18]:
df.shape

(79248, 18)

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
ابعاد ماتریس کاربر-آیتم جدید را نمایش دهید.
</font>
</p>

In [20]:
user_item_matrix = df.pivot_table(index='Applicant ID', columns='Job_ID_Title', values='interaction', fill_value=0)
print(user_item_matrix.shape)
user_item_matrix.head()

(3746, 3267)


Job_ID_Title,100008 specialist artificial intelligence (nlp),100272 designer web,100608 intern developer frontend,100632 intern developer android,100680 developer frontend,101064 developer frontend,101328 software engineer,101376 software engineer frontend,101400 developer frontend,102264 intern developer,...,99240 developer python) backend),99264 php developer,99336 developer حرفه‌ای مwebایل,99624 developer python,99648 developer جوان frontend,99672 developer frontend,99696 developer android,99720 developer frontend,99744 developer frontend,99768 developer backend
Applicant ID,,,,,,,,,,,,,,,,,,,,,
12000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22416.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37872.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58512.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
آیدی 15 کاربری که بیشترین درخواست را داده اند یعنی بیشترین رزومه را برای آگهی ها فرستاده اند همراه با تعداد درخواست ها چاپ کنید.
</font>
</p>

In [21]:
most_applied_users = df['Applicant ID'].value_counts().head(15)
print(most_applied_users)

Applicant ID
10894224.0    198
3902592.0     184
3237456.0     183
5813664.0     147
5105952.0     141
5401248.0     140
2275008.0     136
8049936.0     115
5202816.0     112
3435024.0     110
2674848.0     104
7769856.0     103
5500704.0     101
9985248.0     101
5157168.0      97
Name: count, dtype: int64


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
شباهت سنجی
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
تا به اینجای کار، بحث پیش پردازش داده ها بود و با توجه به داده های یکسان، دقیقا مشابه تمرین قبل بود. اما حالا به قولی دوباره به قلب سیستم های پالایش جمعی رسیدیم. محاسبه شباهت. شما در این مرحله باید یک ماتریس آیتم-آیتم بسازید که سطرها و ستون های این ماتریس، آیتم ها یا تایتل آگهی ها هستند و درایه های ماتریس، میزان شباهت این آیتم ها به یکدیگر را نشان میدهند که با توجه به داده های شما و سوابق محاسبه شده اند. با توجه به اینکه داده ها بصورت 0 و 1 هستند بهتر از معیارهای شباهت مثل <code>Jacard</code> و <code>Dice</code> استفاده کنید. اما با توجه به اینکه اگر از کتابخانه های پایتون و توابع اماده استفاده کنید ابتدا این توابع سعی میکنند ماتریس ها را تبدیل به ماتریس چگال کنند ممکن است از لحاظ میزان استفاده از Ram به مشکل بخورید. پس بهتر است شباهت ژاکارد و دایس را دستی و با توجه به فرمول ها پیاده سازی کنید.
</font>
</p>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
برای محاسبه شباهت بین دو آیتم که با مقادیر دودویی (0 و 1) نمایش داده شده‌اند، می‌توان از معیارهای شباهت <code>Jaccard</code> و <code>Dice</code> استفاده کرد. این معیارها بر پایه مقایسه تعداد اشتراک‌ها و اجتماع ویژگی‌ها بین دو آیتم هستند. فرمول این دو معیار به شکل زیر است:
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
<b>شباهت ژاکارد:</b><br>
</font>
</p>

$$
\text{Jaccard}(A, B) = \frac{|A \cap B|}{|A \cup B|}
$$

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
<b>شباهت دایس:</b><br>
</font>
</p>

$$
\text{Dice}(A, B) = \frac{2 \times |A \cap B|}{|A| + |B|}
$$




In [22]:
def calculate_jaccard_similarity(ui_matrix):
    """
    Calculates the Jaccard similarity matrix between items in a user-item matrix.

    Args:
        ui_matrix: A pandas DataFrame representing the user-item matrix
                   with users as index and items as columns.

    Returns:
        A pandas DataFrame representing the item-item Jaccard similarity matrix.
    """
    # Transpose the matrix to get item-user matrix
    item_user_matrix = ui_matrix.T.values

    num_items = item_user_matrix.shape[0]
    jaccard_sim_matrix = np.zeros((num_items, num_items))

    for i in range(num_items):
        for j in range(i, num_items):
            item_i = item_user_matrix[i]
            item_j = item_user_matrix[j]

            intersection = np.sum(np.logical_and(item_i, item_j))
            union = np.sum(np.logical_or(item_i, item_j))

            if union == 0:
                jaccard_sim_matrix[i, j] = 0.0
            else:
                jaccard_sim_matrix[i, j] = intersection / union

            jaccard_sim_matrix[j, i] = jaccard_sim_matrix[i, j] # Symmetric matrix

    jaccard_similarity_array = pd.DataFrame(
        jaccard_sim_matrix,
        index=ui_matrix.columns,
        columns=ui_matrix.columns
    )

    print("✅ محاسبه ماتریس شباهت JACCARD با موفقیت انجام شد.")
    return jaccard_similarity_array

jaccard_similarity_array = calculate_jaccard_similarity(user_item_matrix)

✅ محاسبه ماتریس شباهت JACCARD با موفقیت انجام شد.


In [24]:
jaccard_similarity_array

Job_ID_Title,100008 specialist artificial intelligence (nlp),100272 designer web,100608 intern developer frontend,100632 intern developer android,100680 developer frontend,101064 developer frontend,101328 software engineer,101376 software engineer frontend,101400 developer frontend,102264 intern developer,...,99240 developer python) backend),99264 php developer,99336 developer حرفه‌ای مwebایل,99624 developer python,99648 developer جوان frontend,99672 developer frontend,99696 developer android,99720 developer frontend,99744 developer frontend,99768 developer backend
Job_ID_Title,,,,,,,,,,,,,,,,,,,,,
100008 specialist artificial intelligence (nlp),1.000000,0.000000,0.000000,0.040000,0.030303,0.000000,0.076923,0.000000,0.000000,0.057143,...,0.000000,0.000000,0.0,0.178571,0.000000,0.031250,0.040000,0.000000,0.000000,0.041667
100272 designer web,0.000000,1.000000,0.129032,0.000000,0.076923,0.000000,0.000000,0.000000,0.062500,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.038462,0.000000,0.000000,0.045455,0.000000
100608 intern developer frontend,0.000000,0.129032,1.000000,0.064516,0.078947,0.030303,0.000000,0.000000,0.034483,0.023256,...,0.034483,0.000000,0.0,0.000000,0.000000,0.025641,0.000000,0.000000,0.028571,0.000000
100632 intern developer android,0.040000,0.000000,0.064516,1.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.115385,...,0.000000,0.000000,0.0,0.041667,0.000000,0.041667,0.125000,0.047619,0.000000,0.062500
100680 developer frontend,0.030303,0.076923,0.078947,0.000000,1.000000,0.080000,0.000000,0.166667,0.150000,0.000000,...,0.000000,0.000000,0.0,0.031250,0.040000,0.178571,0.000000,0.071429,0.115385,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99672 developer frontend,0.031250,0.038462,0.025641,0.041667,0.178571,0.040000,0.025000,0.080000,0.100000,0.000000,...,0.000000,0.000000,0.0,0.032258,0.086957,1.000000,0.041667,0.074074,0.166667,0.043478
99696 developer android,0.040000,0.000000,0.000000,0.125000,0.000000,0.000000,0.062500,0.052632,0.000000,0.074074,...,0.000000,0.071429,0.0,0.041667,0.058824,0.041667,1.000000,0.047619,0.105263,0.133333
99720 developer frontend,0.000000,0.000000,0.000000,0.047619,0.071429,0.000000,0.055556,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.157895,0.074074,0.047619,1.000000,0.136364,0.105263


In [25]:
def calculate_dice_similarity(ui_matrix):
    """
    Calculates the Dice similarity matrix between items in a user-item matrix.

    Args:
        ui_matrix: A pandas DataFrame representing the user-item matrix
                   with users as index and items as columns.

    Returns:
        A pandas DataFrame representing the item-item Dice similarity matrix.
    """
    # Convert the user-item matrix to a NumPy array for efficient calculation
    ui_array = ui_matrix.values

    # Transpose the matrix to get item-user matrix
    item_user_array = ui_array.T

    num_items = item_user_array.shape[0]
    dice_sim_matrix = np.zeros((num_items, num_items))

    for i in range(num_items):
        for j in range(i, num_items): # Calculate for upper triangle including diagonal
            item_i = item_user_array[i]
            item_j = item_user_array[j]

            # Calculate intersection (where both are 1)
            intersection = np.sum(np.logical_and(item_i, item_j))

            # Calculate sum of elements in each item vector
            sum_i = np.sum(item_i)
            sum_j = np.sum(item_j)

            denominator = sum_i + sum_j

            if denominator == 0:
                dice_sim_matrix[i, j] = 0.0 # Avoid division by zero
            else:
                dice_sim_matrix[i, j] = (2 * intersection) / denominator

            dice_sim_matrix[j, i] = dice_sim_matrix[i, j] # The matrix is symmetric

    # Convert the NumPy array back to a DataFrame
    dice_similarity_array = pd.DataFrame(
        dice_sim_matrix,
        index=ui_matrix.columns,
        columns=ui_matrix.columns
    )

    print("✅ محاسبه ماتریس شباهت DICE با موفقیت انجام شد.")
    return dice_similarity_array

dice_similarity_array = calculate_dice_similarity(user_item_matrix)

✅ محاسبه ماتریس شباهت DICE با موفقیت انجام شد.


In [26]:
dice_similarity_array

Job_ID_Title,100008 specialist artificial intelligence (nlp),100272 designer web,100608 intern developer frontend,100632 intern developer android,100680 developer frontend,101064 developer frontend,101328 software engineer,101376 software engineer frontend,101400 developer frontend,102264 intern developer,...,99240 developer python) backend),99264 php developer,99336 developer حرفه‌ای مwebایل,99624 developer python,99648 developer جوان frontend,99672 developer frontend,99696 developer android,99720 developer frontend,99744 developer frontend,99768 developer backend
Job_ID_Title,,,,,,,,,,,,,,,,,,,,,
100008 specialist artificial intelligence (nlp),1.000000,0.000000,0.000000,0.076923,0.058824,0.000000,0.142857,0.000000,0.000000,0.108108,...,0.000000,0.000000,0.0,0.303030,0.000000,0.060606,0.076923,0.000000,0.000000,0.080000
100272 designer web,0.000000,1.000000,0.228571,0.000000,0.142857,0.000000,0.000000,0.000000,0.117647,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.074074,0.000000,0.000000,0.086957,0.000000
100608 intern developer frontend,0.000000,0.228571,1.000000,0.121212,0.146341,0.058824,0.000000,0.000000,0.066667,0.045455,...,0.066667,0.000000,0.0,0.000000,0.000000,0.050000,0.000000,0.000000,0.055556,0.000000
100632 intern developer android,0.076923,0.000000,0.121212,1.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.206897,...,0.000000,0.000000,0.0,0.080000,0.000000,0.080000,0.222222,0.090909,0.000000,0.117647
100680 developer frontend,0.058824,0.142857,0.146341,0.000000,1.000000,0.148148,0.000000,0.285714,0.260870,0.000000,...,0.000000,0.000000,0.0,0.060606,0.076923,0.303030,0.000000,0.133333,0.206897,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99672 developer frontend,0.060606,0.074074,0.050000,0.080000,0.303030,0.076923,0.048780,0.148148,0.181818,0.000000,...,0.000000,0.000000,0.0,0.062500,0.160000,1.000000,0.080000,0.137931,0.285714,0.083333
99696 developer android,0.076923,0.000000,0.000000,0.222222,0.000000,0.000000,0.117647,0.100000,0.000000,0.137931,...,0.000000,0.133333,0.0,0.080000,0.111111,0.080000,1.000000,0.090909,0.190476,0.235294
99720 developer frontend,0.000000,0.000000,0.000000,0.090909,0.133333,0.000000,0.105263,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.272727,0.137931,0.090909,1.000000,0.240000,0.190476


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
فرمول های محاسبه شباهت ژاکارد و دایس بسیار بهم شبیه اند؛ با اجرای کد زیر میتوانید شباهت پیرسون بین ماتریس های شباهت ژاکارد و دایس ببینید
</font>
</p>




In [28]:
jaccard_similarity_array.shape

(3267, 3267)

In [29]:
dice_similarity_array.shape

(3267, 3267)

In [31]:
from scipy.stats import pearsonr
def calculate_overall_correlation(df1, df2):
    # اطمینان از هم‌اندازه بودن و ترتیب یکسان سطر و ستون‌ها
    if not df1.shape == df2.shape or not all(df1.index == df2.index) or not all(df1.columns == df2.columns):
        raise ValueError("دیتافریم‌ها باید ابعاد و اندیس‌های یکسان داشته باشند.")

    values1 = df1.values[np.triu_indices_from(df1.values, k=1)]
    values2 = df2.values[np.triu_indices_from(df2.values, k=1)]

    # محاسبه ضریب همبستگی
    correlation, p_value = pearsonr(values1, values2)

    return correlation, p_value

# محاسبه همبستگی برای نتایج شما
try:
    overall_corr, p_value = calculate_overall_correlation(dice_similarity_array, jaccard_similarity_array)
    print(f"✅ همبستگی کلی پیرسون بین ماتریس‌های Dice و Jaccard: {overall_corr:.4f}")
    print(f"p-value: {p_value:.4f}")
except ValueError as e:
    print(f"خطا در محاسبه همبستگی: {e}")

✅ همبستگی کلی پیرسون بین ماتریس‌های Dice و Jaccard: 0.9979
p-value: 0.0000


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
وقتی نتایج شباهت پیرسون نزدیک 1 باشد، یعنی عملا تفاوتی بین دو معیار ژاکارد و دایس وجود ندارد و هر دو نتایج یکسانی تولید میکنند. پس نیازی به استفاده از هر دو وجود ندارد. راه دیگر استفاده از یک معیار دیگر مثل شباهت کسینوسی است. وقتی داده‌های شما صفر و یک هستند، تفاوت اصلی بین ژاکارد و کسینوسی در برخورد آن‌ها با نبودهای مشترک (مواردی که برای هر دو آیتم مقدار صفر است) خلاصه می‌شود. شباهت ژاکارد، نبودهای مشترک (0-0) را کاملاً نادیده می‌گیرد. اما شباهت کسینوسی نبودهای مشترک (0-0) را در محاسبات خود به طور ضمنی لحاظ می‌کند و بیشتر روی نسبت تعاملات تمرکز دارد. به عبارت دیگر ژاکارد به سادگی می‌پرسد: «از بین کل کاربرانی که حداقل برای یکی از این دو موقعیت رزومه فرستاده‌اند، چه کسری برای هر دو فرستاده‌اند؟» اما شباهت کسینوسی مستقیماً به تعداد کل "۱" ها برای هر آیتم نگاه می‌کند. این یعنی اگر یک موقعیت شغلی بسیار محبوب باشد (تعداد رزومه‌های زیاد) و دیگری کمتر محبوب باشد، مخرج کسر بزرگ شده و ممکن است امتیاز شباهت کاهش یابد، حتی اگر تعداد کاربران مشترکشان یکسان باشد. در واقع، کسینوسی بیشتر به الگو و نسبت شباهت دارد تا همپوشانی مطلق.

<br>
با اجرای سلول زیر ماتریس شباهت کسینوسی آیتم ها را هم بسازید.
</font>
</p>




In [32]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr


def calculate_cosine_similarity(ui_matrix):
    item_user_matrix = ui_matrix.T
    cosine_sim_np = cosine_similarity(item_user_matrix)
    similarity_df = pd.DataFrame(
        cosine_sim_np,
        index=ui_matrix.columns,
        columns=ui_matrix.columns
    )

    print("✅ محاسبه ماتریس شباهت COSINE با موفقیت انجام شد.")
    return similarity_df

item_similarity_cosine = calculate_cosine_similarity(user_item_matrix)

✅ محاسبه ماتریس شباهت COSINE با موفقیت انجام شد.


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
با استفاده از توابع قبل تر نوشته شده، شباهت پیرسون ماتریس های خروجی کسینوسی و ژاکارد را بسنجید
</font>
</p>




In [33]:
# محاسبه شباهت پیرسون بین کسینوسی و ژاکارد

try:
    overall_corr, p_value = calculate_overall_correlation(item_similarity_cosine, jaccard_similarity_array)
    print(f"✅ همبستگی کلی پیرسون بین ماتریس‌های Cosine و Jaccard: {overall_corr:.4f}")
    print(f"p-value: {p_value:.4f}")
except ValueError as e:
    print(f"خطا در محاسبه همبستگی: {e}")

✅ همبستگی کلی پیرسون بین ماتریس‌های Cosine و Jaccard: 0.9901
p-value: 0.0000


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت ششم
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
ماتریس های بدست آمده را به دو دیتافریم تبدیل کنید که یکی برای ژاکارد و دیگری برای کسینوسی باشد. همانطور که گفته شده سطرها و ستونهای دیتافریم عنوان یا تایتل آگهی های شغلی است و مقادیر داخل دیتافریم میزان شباهت این آگهی ها به یکدیگر.
</font>
</p>


In [36]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
## پیشنهاد میشود در این مرحله با توجه به زمان طولانی که صرف شباهت سنجی شده و اینکه در آینده متحمل این حجم نشید؛ دیتافریم مرحله قبل را در سیستم خود ذخیره کنید.

jaccard_similarity_df = jaccard_similarity_array
cosine_similarity_df = item_similarity_cosine

# Save the dataframes in google drive

jaccard_similarity_df.to_csv('/content/drive/MyDrive/Colab Notebooks/jaccard_similarity_df.csv')
cosine_similarity_df.to_csv('/content/drive/MyDrive/Colab Notebooks/cosine_similarity_df.csv')


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت هفتم
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
حالا شما دو دیتافریم یا مجموعه داده دارید که شباهت آگهی ها را با دو رویکرد متفاوت محاسبه کرده اند. در سلول پایین یک تابع پیاده سازی کنید که آیدی کاربر و نام معیار شباهت سنجی (Jaccard یا Cosine) را دریافت کرده و 10 آگهی پیشنهادی را با توجه به معیار شباهت پاس داده شده چاپ کند.

<br>
دیتافریم خروجی شما باید شامل دو ستون <code>Job_ID_Title</code> و <code>Job Domain</code> باشد.

</font>
</p>


In [47]:
def recommender_system(id, metric_df): # metric should be Jaccard or Cosine

    if id not in user_item_matrix.index:
        print(f"خطا: کاربر با شناسه '{id}' در ماتریس کاربر-آیتم یافت نشد.")
        return []

    if metric_df == 'Jaccard':
        similarity_df = jaccard_similarity_df
    elif metric_df == 'Cosine':
        similarity_df = cosine_similarity_df
    else:
        print(f"خطا: معیار شباهت '{metric_df}' نامعتبر است. لطفا 'Jaccard' یا 'Cosine' را انتخاب کنید.")
        return []

    user_jobs = user_item_matrix.loc[id]
    user_jobs = user_jobs[user_jobs > 0]
    user_jobs = user_jobs.index.tolist() # Convert index to list
    other_jobs = list(set(similarity_df.columns) - set(user_jobs)) # Convert set to list
    similarity_scores = similarity_df.loc[other_jobs, user_jobs]
    similarity_scores = similarity_scores.mean(axis=1)
    similarity_scores = similarity_scores.sort_values(ascending=False)
    top_jobs = similarity_scores.head(10)

    # Get Job Domain for recommended jobs
    recommended_jobs_info = df[df['Job_ID_Title'].isin(top_jobs.index)][['Job_ID_Title', 'Job Domain']].drop_duplicates().reset_index(drop=True)

    return recommended_jobs_info


user_id = user_item_matrix.index[0]
top_jobs_jaccard = recommender_system(user_id, 'Jaccard')
print("Top 10 recommendations using Jaccard similarity:")
print(top_jobs_jaccard)

user_id_2 = user_item_matrix.index[1]
top_jobs_cosine = recommender_system(user_id_2, 'Cosine')
print("\nTop 10 recommendations using Cosine similarity:")
print(top_jobs_cosine)

Top 10 recommendations using Jaccard similarity:
                               Job_ID_Title Job Domain
0                  545544 developer backend    Backend
1                  541728 developer backend    Backend
2                  539808 software engineer      Other
3         540264 (backend developer (django    Backend
4         544104 developer (backend (python    Backend
5        542448 software engineer (backend)    Backend
6                  545256 developer backend    Backend
7  537984 (senior backend developer (django    Backend
8                  539904 developer backend    Backend
9                  541080 developer backend    Backend

Top 10 recommendations using Cosine similarity:
                                  Job_ID_Title Job Domain
0              462624 python backend developer    Backend
1  466056 django internship (remote or hybrid)    Backend
2     346992 developer junior backend (django)    Backend
3     465552 developer junior backend (django)    Backend
4  4273

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت هشت
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
تابع زیر را اجرا کنید. تابع زیر یک ایدی کاربران را دریافت میکند و 15 مورد از اگهی هایی که یک فرد رزومه ارسال کرده را نمایش میدهد. میتوانید با آیدی های متفاوت تابع را تست کنید.
</font>
</p>


In [49]:
def get_user_applied_jobs(user_id, num_jobs_to_return=15, user_item_matrix_df=user_item_matrix):
    if user_id not in user_item_matrix_df.index:
        print(f"خطا: کاربر با شناسه '{user_id}' در ماتریس کاربر-آیتم یافت نشد.")
        return []

    # انتخاب ردیف مربوط به کاربر هدف
    user_interactions_series = user_item_matrix_df.loc[user_id]

    # فیلتر کردن آیتم‌هایی که کاربر با آن‌ها تعامل داشته (مقدارشان 1.0 یا بیشتر است)
    applied_jobs_series = user_interactions_series[user_interactions_series >= 1.0]

    # استخراج نام آیتم‌ها (که همان نام ستون‌ها در ردیف انتخاب شده است)
    applied_jobs_list = applied_jobs_series.index.tolist()

    # برگرداندن حداکثر num_jobs_to_return شغل
    return applied_jobs_list[:num_jobs_to_return]

get_user_applied_jobs(user_id=3902592.0)

['100272 designer web',
 '102792 intern developer net.',
 '106104 developer frontend',
 '108240 intern react native',
 '108264 intern frontend',
 '109632 developer frontend',
 '111048 intern developer front end',
 '112944 developer backend',
 '115800 developer react) frontend)',
 '118056 intern react',
 '121056 intern frontend',
 '123336 intern developer web',
 '123936 کاز آموز developer frontend',
 '134064 intern developer web',
 '150072 react js frontend developer']

In [50]:
user_id = 3902592.0
top_jobs_jaccard = recommender_system(user_id, 'Jaccard')
print("Top 10 recommendations using Jaccard similarity:")
print(top_jobs_jaccard)

Top 10 recommendations using Jaccard similarity:
                          Job_ID_Title Job Domain
0            315144 developer frontend   Frontend
1    314088 developer frontend (react)   Frontend
2            333408 developer frontend   Frontend
3            306264 developer react js   Frontend
4            278952 developer frontend   Frontend
5            295560 developer frontend   Frontend
6  311976 developer frontend و reactjs   Frontend
7            286824 developer frontend   Frontend
8        291984 توسعه‌‌دهنده‌ frontend   Frontend
9            280152 developer frontend   Frontend


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت هشت ممیز یک
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
خب شما حالا انگار یک ماشین دارید که با دو موتور متفاوت کار میکند. در سلول های زیر، از شما خواسته شده تا برای کاربران متفاوت لیست پیشنهادی شامل 10 آگهی شغلی تولید کنید. اینکه از کدام یک از موتورهای ژاکارد یا کسینوسی استفاده کنید بر عهده شماست. در نهایت هم ارزیابی به این شکل انجام میشود که <code>تعداد آیتم های هم حوزه</code> شمرده میشوند. (precision@10 مشابه پروژه 2)

<br>
دیتافریم خروجی شما باید شامل دو ستون <code>Job_ID_Title</code> و <code>Job Domain</code> باشد.

</font>
</p>


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت هشت ممیز دو
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
با استفاده از تابعی که نوشتید و معیار ارزیابی که فکر میکنید بیشترین دقت را خواهد داشت برای کاربر <code>3902592.0</code> لیست پیشنهادی 10 تایی شامل ستون های نامبرده شده را تولید کنید و نتیجه را در یک فایل csv به نام <code>3902592.csv</code> ذخیره کنید.
</font>
</p>


In [55]:
def precision_at_k(recommended_jobs_domains, user_applied_jobs, k=10):
    """
    Calculates Precision@k based on Job Domain overlap.

    Args:
        recommended_jobs_domains: A list or pandas Series of Job Domains
                                  for the recommended jobs.
        user_applied_jobs: A list of Job_ID_Title strings for jobs the user applied to.
        k: The number of top recommendations to consider.

    Returns:
        The Precision@k score.
    """
    user_applied_jobs_domains = df[df['Job_ID_Title'].isin(user_applied_jobs)]['Job Domain'].tolist()

    # Count how many of the recommended job domains are in the user's applied job domains
    relevant_recommendations = sum(domain in user_applied_jobs_domains for domain in recommended_jobs_domains[:k])

    # Calculate precision@k
    precision = relevant_recommendations / k if k > 0 else 0

    return precision


user_id = 3902592.0
top_jobs_jaccard = recommender_system(user_id, 'Jaccard')
user_applied_jobs = get_user_applied_jobs(user_id, num_jobs_to_return=10)
print("Top 10 recommendations using Jaccard similarity:")
print(top_jobs_jaccard)
print("get user applied jobs:")
print(user_applied_jobs)
print("Precision@10:", precision_at_k(top_jobs_jaccard['Job Domain'], user_applied_jobs))

Top 10 recommendations using Jaccard similarity:
                          Job_ID_Title Job Domain
0            315144 developer frontend   Frontend
1    314088 developer frontend (react)   Frontend
2            333408 developer frontend   Frontend
3            306264 developer react js   Frontend
4            278952 developer frontend   Frontend
5            295560 developer frontend   Frontend
6  311976 developer frontend و reactjs   Frontend
7            286824 developer frontend   Frontend
8        291984 توسعه‌‌دهنده‌ frontend   Frontend
9            280152 developer frontend   Frontend
get user applied jobs:
['100272 designer web', '102792 intern developer net.', '106104 developer frontend', '108240 intern react native', '108264 intern frontend', '109632 developer frontend', '111048 intern developer front end', '112944 developer backend', '115800 developer react) frontend)', '118056 intern react']
Precision@10: 1.0


In [56]:
user_id = 3902592.0
top_jobs_cosine = recommender_system(user_id, 'Cosine')
user_applied_jobs = get_user_applied_jobs(user_id, num_jobs_to_return=10)
print("Top 10 recommendations using Cosine similarity:")
print(top_jobs_cosine)
print("get user applied jobs:")
print(user_applied_jobs)
print("Precision@10:", precision_at_k(top_jobs_cosine['Job Domain'], user_applied_jobs))

Top 10 recommendations using Cosine similarity:
                          Job_ID_Title Job Domain
0            315144 developer frontend   Frontend
1    314088 developer frontend (react)   Frontend
2            333408 developer frontend   Frontend
3            306264 developer react js   Frontend
4            278952 developer frontend   Frontend
5            295560 developer frontend   Frontend
6  311976 developer frontend و reactjs   Frontend
7            286824 developer frontend   Frontend
8        291984 توسعه‌‌دهنده‌ frontend   Frontend
9            280152 developer frontend   Frontend
get user applied jobs:
['100272 designer web', '102792 intern developer net.', '106104 developer frontend', '108240 intern react native', '108264 intern frontend', '109632 developer frontend', '111048 intern developer front end', '112944 developer backend', '115800 developer react) frontend)', '118056 intern react']
Precision@10: 1.0


In [51]:
user_id = 3902592.0
top_jobs_cosine = recommender_system(user_id, 'Cosine')
print("\nTop 10 recommendations using Cosine similarity:")
print(top_jobs_cosine)
top_jobs_cosine.to_csv('3902592.csv', index=False)


Top 10 recommendations using Cosine similarity:
                          Job_ID_Title Job Domain
0            315144 developer frontend   Frontend
1    314088 developer frontend (react)   Frontend
2            333408 developer frontend   Frontend
3            306264 developer react js   Frontend
4            278952 developer frontend   Frontend
5            295560 developer frontend   Frontend
6  311976 developer frontend و reactjs   Frontend
7            286824 developer frontend   Frontend
8        291984 توسعه‌‌دهنده‌ frontend   Frontend
9            280152 developer frontend   Frontend


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت هشت ممیز سه
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
با استفاده از تابعی که نوشتید و معیار ارزیابی که فکر میکنید بیشترین دقت را خواهد داشت برای کاربر <code>5401248.0</code> لیست پیشنهادی 10 تایی شامل ستون های نامبرده شده را تولید کنید و نتیجه را در یک فایل csv به نام <code>5401248.csv</code> ذخیره کنید.
</font>
</p>



In [58]:
user_id = 5401248.0
print("get user applied jobs:")
print(get_user_applied_jobs(user_id, num_jobs_to_return=10))
top_jobs_jaccard = recommender_system(user_id, 'Jaccard')
top_jobs_cosine = recommender_system(user_id, 'Cosine')
print("Top 10 recommendations using Jaccard similarity:")
print(top_jobs_jaccard)
print("Top 10 recommendations using Cosine similarity:")
print(top_jobs_cosine)
precision_jaccard = precision_at_k(top_jobs_jaccard['Job Domain'], get_user_applied_jobs(user_id, num_jobs_to_return=10))
print("Precision@10 using Jaccard similarity:", precision_jaccard)
precision_cosine = precision_at_k(top_jobs_cosine['Job Domain'], get_user_applied_jobs(user_id, num_jobs_to_return=10))
print("Precision@10 using Cosine similarity:", precision_cosine)

get user applied jobs:
['102264 intern developer', '102576 developer web', '103656 developer android', '103992 کار آموز python', '108216 کار آموز php', '247392 developer python) backend)', '260976 developer python (python/django)', '262152 python/django developer', '296640 developer backend (python/django)', '311280 backend developer']
Top 10 recommendations using Jaccard similarity:
                               Job_ID_Title Job Domain
0                      380856 django intern    Backend
1                   365928 developer django    Backend
2                  351000 software engineer      Other
3                   340344 python developer    Backend
4            338496 developer python python    Backend
5        409584 software engineer (backend)    Backend
6                  342168 backend developer    Backend
7  386808 developer junior backend (django)    Backend
8                   377112 developer python    Backend
9                   408840 developer django    Backend
Top 10 r

In [59]:
top_jobs_cosine.to_csv('5401248.csv', index=False)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
قسمت هشت ممیز چهار
</font>
</h2>
<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
با استفاده از تابعی که نوشتید و معیار ارزیابی که فکر میکنید بیشترین دقت را خواهد داشت برای کاربر <code>66624.0</code> لیست پیشنهادی 10 تایی شامل ستون های نامبرده شده را تولید کنید و نتیجه را در یک فایل csv به نام <code>66624.csv</code> ذخیره کنید.
</font>
</p>

In [60]:
user_id = 66624.0
print("get user applied jobs:")
print(get_user_applied_jobs(user_id, num_jobs_to_return=10))
top_jobs_jaccard = recommender_system(user_id, 'Jaccard')
top_jobs_cosine = recommender_system(user_id, 'Cosine')
print("Top 10 recommendations using Jaccard similarity:")
print(top_jobs_jaccard)
print("Top 10 recommendations using Cosine similarity:")
print(top_jobs_cosine)
precision_jaccard = precision_at_k(top_jobs_jaccard['Job Domain'], get_user_applied_jobs(user_id, num_jobs_to_return=10))
print("Precision@10 using Jaccard similarity:", precision_jaccard)
precision_cosine = precision_at_k(top_jobs_cosine['Job Domain'], get_user_applied_jobs(user_id, num_jobs_to_return=10))
print("Precision@10 using Cosine similarity:", precision_cosine)
top_jobs_cosine.to_csv('66624.csv', index=False)

get user applied jobs:
['278352 developer android intern', '311472 developer android (android developer)', '311568 developer android (android developer)', '314304 android developer', '316632 developer android android developer', '351144 developer مwebایل (android)', '351840 developer مwebایل (android)', '352008 android developer', '353952 android developer', '359280 android developer']
Top 10 recommendations using Jaccard similarity:
                        Job_ID_Title Job Domain
0           144408 developer android    Android
1           143400 developer android    Android
2    337824 senior android developer    Android
3           311448 developer android    Android
4           342096 developer android    Android
5       354360 android sdk developer    Android
6  357576 android developer snappbox    Android
7           358824 android developer    Android
8           329040 developer android    Android
9           350184 android developer    Android
Top 10 recommendations using Cosin

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>

In [ ]:
import zipfile

def compress(file_names):
    print("File Paths:")
    print(file_names)
    # Select the compression mode ZIP_DEFLATED for compression
    # or zipfile.ZIP_STORED to just store the file
    compression = zipfile.ZIP_DEFLATED
    # create the zip file first parameter path/name, second mode
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            # Add file to the zip file
            # first parameter file to zip, second filename in zip
            zf.write('./' + file_name, file_name, compress_type=compression)


file_names = ["notebook.ipynb", "3902592.csv",  "5401248.csv", "66624.csv"]
compress(file_names)